### Ex-post Efficiency
1. Decompose a biostochastic matrix using Birkhoff algorithm. 
2. Calculate the efficiency violation of each permutatioin matrix. 
3. Calculate the weighted average of the violation. 

cf. [Birkhoff Algorithm](https://en.wikipedia.org/wiki/Birkhoff_algorithm)

In [ ]:
def build_graph(self, Q):
        graph = {a: [] for a in range(self.n)}
        for i in range(self.n):
            sorted_preferences = torch.argsort(self.preferences[i], descending=True)
            for a in sorted_preferences:
                for b in sorted_preferences:
                    if a == b:
                        continue
                    if Q[i, b] > 0:
                        if a not in graph:
                            graph[a] = []
                        graph[a].append((b, i, Q[i, b].item()))
                        break
        return graph
    
    def find_cycle(self, graph):
        visited = set()
        rec_stack = []

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                if nbr in visited:
                    cycle_edges = [(v, nbr, agent, avail)]
                    for node, edge in reversed(rec_stack):
                        if node == nbr:
                            break
                        cycle_edges.append(edge)
                    return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None
    
    def execute_all_cycles(self):
        Q = self.P.clone()
        cycles_exchanges = []
        violation = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            violation += epsilon
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

In [ ]:
import torch
import numpy as np

class compute_ev:
    def __init__(self, cfg, P, preferences):
        """
        P: n*n の二重確率行列 (torch.Tensor)
        preferences: n*n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.cfg = cfg
        self.P = P.clone()
        if isinstance(preferences, np.ndarray):
            preferences = torch.tensor(preferences, dtype=torch.float32)
        self.preferences = preferences.clone().detach().float()
        self.n = cfg.num_goods

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and Q[i, b] > 0:
                        graph[a].append((b, i, Q[i, b].item())) 
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        violation = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            violation += epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

    def execute_all_cycles_batch(self):
        """
        P と preferences の各バッチに対して execute_all_cycles を計算し、
        それらの結果を合わせて n*1 の行列にする。
        """
        batch_size = self.P.shape[0]
        results = torch.zeros((batch_size, 1), dtype=torch.float32)

        for b in range(batch_size):
            P_batch = self.P[b].view(self.n, self.n)
            preferences_batch = self.preferences[b].view(self.n, self.n)
            ev_instance = compute_ev(self.cfg, P_batch, preferences_batch)
            results[b, 0] = ev_instance.execute_all_cycles()

        return results

In [ ]:
import torch
import numpy as np

class compute_ev:
    def __init__(self, cfg, P, preferences):
        """
        P: batch_size*n*n の二重確率行列 (torch.Tensor)
        preferences: batch_size*n*n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.cfg = cfg
        self.P = P.clone().to(P.device)  # ★変更箇所: P のデバイス維持
        if isinstance(preferences, np.ndarray):
            preferences = torch.tensor(preferences, dtype=torch.float32)
        self.preferences = preferences.clone().detach().float().to(self.P.device)  # ★変更箇所: preferences を P と同じデバイスに
        self.n = cfg.num_goods

    def vectorized_build_graph(self, Q):
        """
        ★変更箇所: グラフ構築をテンソル演算で実施  
        各 (a, b) ペアについて、条件 (preferences[i,a] > preferences[i,b] かつ Q[i,b] > 0)
        をブロードキャストで一括評価し、条件を満たす最初の witness agent とその Q 値を求めます。
        """
        n = self.n
        # self.preferences, Q は同一デバイス上にある前提
        cond = (self.preferences.unsqueeze(2) > self.preferences.unsqueeze(1)) & (Q.unsqueeze(1) > 0)
        exists = cond.any(dim=0)  # shape: (n, n)
        witness = torch.argmax(cond.float(), dim=0)  # shape: (n, n)
        witness = torch.where(exists, witness, torch.full_like(witness, -1))
        goods_idx = torch.arange(n, device=Q.device).view(1, n).expand(n, n)
        V = torch.where(witness != -1, Q[witness.clamp(min=0), goods_idx], torch.zeros_like(witness, dtype=Q.dtype))
        A = (V > 0).float()
        A = A * (1 - torch.eye(n, device=Q.device))
        return witness, V, A

    def vectorized_execute_all_cycles(self, max_cycle_length=4, max_iter=10):
        """
        ★変更箇所:  
         - 動的な while ループと .item() 呼び出しの代わりに、固定回数の for ループで反復処理を行います。
         - これにより、vmap 内での動的制御フローや .item() によるスカラー化を回避します。
         - 各反復で、2サイクル、3サイクル、4サイクルをすべて検出し、対応する更新を accumulate します。
        """
        n = self.n
        device = self.P.device
        Q = self.P.clone()
        total_violation = torch.tensor(0.0, device=device)

        for _ in range(max_iter):  # ★変更箇所: while ループを固定回数の for ループに変更
            witness, V, A = self.vectorized_build_graph(Q)
            update = torch.zeros_like(Q)
            cycle_violation = torch.tensor(0.0, device=device)

            # ----- 2サイクル検出・更新 -----
            # ★変更箇所: 固定シェイプのインデックスを torch.triu_indices で取得
            idx2 = torch.triu_indices(n, n, offset=1, device=device)  # shape: (2, num_pairs)
            a_idx2 = idx2[0]  # shape: (num_pairs,)
            b_idx2 = idx2[1]
            valid_2 = ((A[a_idx2, b_idx2] > 0) & (A[b_idx2, a_idx2] > 0)).float()  # 0/1 のブールマスク
            eps_2 = torch.min(V[a_idx2, b_idx2], V[b_idx2, a_idx2]) * valid_2  # 条件を満たさない箇所は 0
            cycle_violation = cycle_violation + eps_2.sum()  # ★.item() を使用せず tensor のまま加算
            w_ab = witness[a_idx2, b_idx2]
            w_ba = witness[b_idx2, a_idx2]
            update.index_put_((w_ab, b_idx2), -eps_2, accumulate=True)
            update.index_put_((w_ab, a_idx2),  eps_2, accumulate=True)
            update.index_put_((w_ba, a_idx2), -eps_2, accumulate=True)
            update.index_put_((w_ba, b_idx2),  eps_2, accumulate=True)

            # ----- 3サイクル検出・更新 -----
            if max_cycle_length >= 3:
                # ★変更箇所: 全 triplet (a,b,c) を固定シェイプで生成（flatten して使用）
                grid3 = torch.meshgrid(torch.arange(n, device=device),
                                       torch.arange(n, device=device),
                                       torch.arange(n, device=device),
                                       indexing='ij')
                a_idx3 = grid3[0].flatten()  # shape: (n^3,)
                b_idx3 = grid3[1].flatten()
                c_idx3 = grid3[2].flatten()
                # 各 triplet がすべて異なる条件（0/1 のマスク）
                distinct3 = ((a_idx3 != b_idx3) & (b_idx3 != c_idx3) & (a_idx3 != c_idx3)).float()
                cond3 = ((A[a_idx3, b_idx3] > 0) & (A[b_idx3, c_idx3] > 0) & (A[c_idx3, a_idx3] > 0)).float()
                valid_3 = distinct3 * cond3
                eps_3 = torch.min(torch.min(V[a_idx3, b_idx3], V[b_idx3, c_idx3]), V[c_idx3, a_idx3])
                eps_3 = eps_3 * valid_3
                cycle_violation = cycle_violation + eps_3.sum()
                w_ab_3 = witness[a_idx3, b_idx3]
                w_bc_3 = witness[b_idx3, c_idx3]
                w_ca_3 = witness[c_idx3, a_idx3]
                update.index_put_((w_ab_3, b_idx3), -eps_3, accumulate=True)
                update.index_put_((w_ab_3, a_idx3),  eps_3, accumulate=True)
                update.index_put_((w_bc_3, c_idx3), -eps_3, accumulate=True)
                update.index_put_((w_bc_3, b_idx3),  eps_3, accumulate=True)
                update.index_put_((w_ca_3, a_idx3), -eps_3, accumulate=True)
                update.index_put_((w_ca_3, c_idx3),  eps_3, accumulate=True)

            # ----- 4サイクル検出・更新 -----
            if max_cycle_length >= 4:
                # ★変更箇所: 全 quadruplet (a,b,c,d) を固定シェイプで生成
                grid4 = torch.meshgrid(torch.arange(n, device=device),
                                       torch.arange(n, device=device),
                                       torch.arange(n, device=device),
                                       torch.arange(n, device=device),
                                       indexing='ij')
                a_idx4 = grid4[0].flatten()  # shape: (n^4,)
                b_idx4 = grid4[1].flatten()
                c_idx4 = grid4[2].flatten()
                d_idx4 = grid4[3].flatten()
                # すべてのノードが異なる条件（0/1 マスク）
                distinct4 = ((a_idx4 != b_idx4) & (a_idx4 != c_idx4) & (a_idx4 != d_idx4) &
                             (b_idx4 != c_idx4) & (b_idx4 != d_idx4) & (c_idx4 != d_idx4)).float()
                cond4 = ((A[a_idx4, b_idx4] > 0) & (A[b_idx4, c_idx4] > 0) & 
                         (A[c_idx4, d_idx4] > 0) & (A[d_idx4, a_idx4] > 0)).float()
                valid_4 = distinct4 * cond4
                eps_4 = torch.min(torch.min(torch.min(V[a_idx4, b_idx4], V[b_idx4, c_idx4]), V[c_idx4, d_idx4]), V[d_idx4, a_idx4])
                eps_4 = eps_4 * valid_4
                cycle_violation = cycle_violation + eps_4.sum()
                w_ab_4 = witness[a_idx4, b_idx4]
                w_bc_4 = witness[b_idx4, c_idx4]
                w_cd_4 = witness[c_idx4, d_idx4]
                w_da_4 = witness[d_idx4, a_idx4]
                update.index_put_((w_ab_4, b_idx4), -eps_4, accumulate=True)
                update.index_put_((w_ab_4, a_idx4),  eps_4, accumulate=True)
                update.index_put_((w_bc_4, c_idx4), -eps_4, accumulate=True)
                update.index_put_((w_bc_4, b_idx4),  eps_4, accumulate=True)
                update.index_put_((w_cd_4, d_idx4), -eps_4, accumulate=True)
                update.index_put_((w_cd_4, c_idx4),  eps_4, accumulate=True)
                update.index_put_((w_da_4, a_idx4), -eps_4, accumulate=True)
                update.index_put_((w_da_4, d_idx4),  eps_4, accumulate=True)

            # ★変更箇所: 固定反復回数のため、更新がなくてもループは max_iter 回実行
            Q = Q + update
            total_violation = total_violation + cycle_violation

        return total_violation

    def execute_all_cycles(self):
        """
        参考用の逐次処理版
        """
        Q = self.P.clone()
        cycles_exchanges = []
        violation = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            violation += epsilon
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

    def execute_all_cycles_batch(self):
        """
        ★変更箇所:  
         - torch.vmap を使用して各バッチを一斉に処理  
         - vectorized_execute_all_cycles を vmap で呼び出す  
         ※ self.P と self.preferences のバッチサイズが異なる場合は（例: preferences が2倍なら先頭半分を採用）調整します。
        """
        def process_single(P_batch, preferences_batch):
            ev_instance = compute_ev(self.cfg, P_batch, preferences_batch)
            return ev_instance.vectorized_execute_all_cycles(max_cycle_length=4, max_iter=10)

        if self.P.shape[0] != self.preferences.shape[0]:
            if self.preferences.shape[0] == 2 * self.P.shape[0]:
                fixed_preferences = self.preferences[:self.P.shape[0]]
            else:
                raise ValueError(
                    f"Batch size mismatch: self.P shape {self.P.shape} vs self.preferences shape {self.preferences.shape}. "
                    "These must match."
                )
        else:
            fixed_preferences = self.preferences

        results = torch.vmap(process_single)(self.P, fixed_preferences)
        return results.unsqueeze(-1)


In [ ]:
import torch
import numpy as np

class compute_ev:
    def __init__(self, cfg, P, preferences):
        """
        P: batch_size*n*n の二重確率行列 (torch.Tensor)
        preferences: batch_size*n*n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.cfg = cfg
        self.P = P.clone()
        if isinstance(preferences, np.ndarray):
            preferences = torch.tensor(preferences, dtype=torch.float32)
        self.preferences = preferences.clone().detach().float()
        self.n = cfg.num_goods

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and Q[i, b] > 0:
                        graph[a].append((b, i, Q[i, b].item())) 
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        violation = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            violation += epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

    def execute_all_cycles_batch(self):

        # 単一バッチ（1要素）に対してexecute_all_cyclesを実行する関数を定義
        def process_single(P_single, preferences_single):
            # compute_evのインスタンスを生成（各バッチは独立して処理）
            ev_instance = compute_ev(self.cfg, P_single, preferences_single)
            # 各バッチ内でのサイクル交換を実行し、その累積交換量（violation）を返す
            return ev_instance.execute_all_cycles()

        # vmapを用いて、Pおよびpreferencesの各バッチに対してprocess_singleを並列実行
        results = torch.vmap(process_single)(self.P, self.preferences)
        # 結果は1次元（バッチサイズ）なので、n*1の行列形式に整形して返す
        return results.unsqueeze(1)


In [ ]:
import torch
import numpy as np

class compute_ev:
    def __init__(self, cfg, P, preferences):
        """
        P: batch_size*n*n の二重確率行列 (torch.Tensor)
        preferences: batch_size*n*n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.cfg = cfg
        self.P = P.clone()
        if isinstance(preferences, np.ndarray):
            preferences = torch.tensor(preferences, dtype=torch.float32)
        self.preferences = preferences.clone().detach().float()
        self.n = cfg.num_goods

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if (self.preferences[i, a] > self.preferences[i, b]).item() and (Q[i, b] > 0).item():
                        graph[a].append((b, i, Q[i, b].item())) 
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        violation = torch.tensor(0.0)
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = torch.tensor([edge[3] for edge in cycle])
            epsilon = torch.min(epsilons)
            violation += epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

    def execute_all_cycles_batch(self):
        """
        バッチ全体に対して execute_all_cycles を並列実行する。
        """
        results = torch.stack([process_single(self.cfg, P_single, preferences_single) for P_single, preferences_single in zip(self.P, self.preferences)])
        return results.unsqueeze(1)

def process_single(cfg, P_single, preferences_single):
    """
    単一バッチ（1要素）に対して execute_all_cycles を実行する関数。
    """
    ev_instance = compute_ev(cfg, P_single, preferences_single)
    return ev_instance.execute_all_cycles()

In [ ]:
import torch
import numpy as np

class compute_ev:
    def __init__(self, cfg, P, preferences):
        """
        P: batch_size*n*n の二重確率行列 (torch.Tensor)
        preferences: batch_size*n*n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.cfg = cfg
        self.device = P.device
        self.P = P.clone().to(self.device)
        if isinstance(preferences, np.ndarray):
            preferences = torch.tensor(preferences, dtype=torch.float32)
        self.preferences = preferences.clone().detach().float().to(self.device)
        self.n = cfg.num_goods

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        preferences = self.preferences
        Q = Q.detach()

        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                mask = (preferences[:, a] > preferences[:, b]) & (Q[:, b] > 0)
                if mask.any():
                    i = mask.nonzero(as_tuple=True)[0][0]
                    graph[a].append((b, i, Q[i, b].item()))
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        violation = torch.tensor(0.0, device=self.device)
        with torch.no_grad():  # 勾配計算を無効にする
            while True:
                graph = self.build_graph(Q)
                cycle = self.find_cycle(graph)
                if cycle is None:
                    break
                epsilons = torch.tensor([edge[3] for edge in cycle], device=self.device)
                epsilon = torch.min(epsilons)
                violation += epsilon
                for (a, b, agent, avail) in cycle:
                    Q[agent, b] -= epsilon
                    Q[agent, a] += epsilon
                cycles_exchanges.append((cycle, epsilon))
        return violation

    def execute_all_cycles_batch(self):
        """
        バッチ全体に対して execute_all_cycles を並列実行する。
        """
        results = torch.stack([process_single(self.cfg, P_single, preferences_single) for P_single, preferences_single in zip(self.P, self.preferences)])
        return results.unsqueeze(1)

def process_single(cfg, P_single, preferences_single):
    """
    単一バッチ（1要素）に対して execute_all_cycles を実行する関数。
    """
    ev_instance = compute_ev(cfg, P_single, preferences_single)
    return ev_instance.execute_all_cycles()

In [ ]:
import torch
import numpy as np
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor

class compute_ev:
    def __init__(self, cfg, P, preferences):
        """
        P: n*n の二重確率行列 (torch.Tensor)
        preferences: n*n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.cfg = cfg
        self.P = P.clone()
        if isinstance(preferences, np.ndarray):
            preferences = torch.tensor(preferences, dtype=torch.float32)
        self.preferences = preferences.clone().detach().float()
        self.n = cfg.num_goods

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and Q[i, b] > 0:
                        graph[a].append((b, i, Q[i, b].item())) 
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        violation = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            violation += epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

    def process_batch(self, b):
        P_batch = self.P[b].detach().clone().view(self.n, self.n)
        preferences_batch = self.preferences[b].detach().clone().view(self.n, self.n)
        ev_instance = compute_ev(self.cfg, P_batch, preferences_batch)
        return ev_instance.execute_all_cycles()

    def execute_all_cycles_batch(self):
        """
        P と preferences の各バッチに対して execute_all_cycles を計算し、
        それらの結果を合わせて n*1 の行列にする。
        """
        batch_size = self.P.shape[0]
        results = torch.zeros((batch_size, 1), dtype=torch.float32)

        with ProcessPoolExecutor() as executor:
            futures = [executor.submit(self.process_batch, b) for b in range(batch_size)]
            for b, future in enumerate(concurrent.futures.as_completed(futures)):
                results[b, 0] = future.result()

        return results

In [ ]:
import torch
import numpy as np
import multiprocessing as mp

class compute_ev:
    def __init__(self, cfg, P, preferences):
        """
        P: n*n の二重確率行列 (torch.Tensor)
        preferences: n*n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.cfg = cfg
        self.P = P.clone().detach()
        if isinstance(preferences, np.ndarray):
            preferences = torch.tensor(preferences, dtype=torch.float32)
        self.preferences = preferences.clone().detach().float()
        self.n = cfg.num_goods

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and Q[i, b] > 0:
                        graph[a].append((b, i, Q[i, b].item())) 
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        violation = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            violation += epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

    def execute_all_cycles_batch(self):
        """
        P と preferences の各バッチに対して execute_all_cycles を計算し、
        それらの結果を合わせて n*1 の行列にする。
        """
        batch_size = self.P.detach().shape[0]
        results = torch.zeros((batch_size, 1), dtype=torch.float32)

        pool = mp.Pool(mp.cpu_count())
        results = pool.map(self.execute_all_cycles, range(batch_size))
    
        return results